In [140]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import copy
import pickle 
data = loadmat('ex3data1.mat')

# print(data.keys())
#
# print(data['X'])
#
# print(len(data['X']))
#
# print(data['y'])
#
# print(len(data['y']))

X = data['X']
Y = data['y']

# temp = np.reshape(X[0], (20, 20))
# plt.imshow(temp)
# plt.show()


for i in range(len(Y)):
    if Y[i][0] == 10:
        Y[i] = 0
    else:
        Y[i] = Y[i][0]

labels = []
for label in Y:
     labels.append(label[0])

X_sparse = coo_matrix(X)

X, X_sparse, labels = shuffle(X, X_sparse, labels, random_state=0)


X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=0)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0)


In [141]:
X_train[0].shape

(400,)

In [183]:
epochs = 2000
lr = 1e-4

In [175]:
# only for one example
class Linear:
    def __init__(self, input_dim, output_dim):
        self.w = np.random.randn(output_dim, input_dim)
        self.b = np.random.randn(output_dim, 1)
        self.x = None
        self.dw = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        
        out = np.dot(self.w, x) + self.b
        
        return out
    
    def backward(self, dout):
        self.dw = np.dot(dout, self.x.T)
        self.db = dout
        
        return np.dot(self.w.T, dout)

In [176]:
# only for one example
class ReLU:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        return dout

In [177]:
# only for one example
class softmax:
    def __init__(self):
        self.p = None
        self.t = None
    
    def forward(self, a, t):
        self.t = t
        c = np.max(a)
        exp_a = np.exp(a - c)
        sum_exp_a = np.sum(exp_a)
        p = exp_a / sum_exp_a
        
        self.p = p
        
        _p = np.log(p + 1e-5)

        loss = np.sum(-_p*t)
        
        return loss
    def backward(self):
        
        return self.p - self.t

In [178]:
model = []
layer_1=Linear(400, 300)
model.append(layer_1)

layer_2 = ReLU()
model.append(layer_2)

layer_3=Linear(300, 200)
model.append(layer_3)

layer_4 = ReLU()
model.append(layer_4)

layer_5=Linear(200, 100)
model.append(layer_5)

layer_6 = ReLU()
model.append(layer_6)

layer_7 = Linear(100, 10)
model.append(layer_7)

layer_8 = softmax()
model.append(layer_8)

for layer in model:
    print(layer)

In [179]:
def MakeOneHot(class_num, x):
    true = np.zeros(class_num)
    true[x] = 1
    return true.reshape(class_num, -1)

In [180]:
class DNN:
    def __init__(self, model):
        self.model = model
    
    def forward(self, x, y):
        for layer in self.model[0:-1]:
            x = layer.forward(x)
        
        x = model[-1].forward(x, y)
        return x
    
    def backward(self):
        dout = model[-1].backward()
        
        index = len(model) - 2
        while index >= 0:
            dout = model[index].backward(dout)
            index -= 1

In [184]:
net = DNN(model=model)
# loss = net.forward(X_train[0].reshape(-1, 1), MakeOneHot(10, y_train[0]))
# net.backward()
# print(loss)

best_model = None
best_acc = -1
for epoch in range(epochs):
    loss = 0
    for i in range(len(X_train)):

        loss += net.forward(X_train[i].reshape(-1, 1), MakeOneHot(10, y_train[i]))
        net.backward()
        
        # update parameters
        index = len(net.model) - 2
        while index >= 0:
            net.model[index].w -= lr * net.model[index].dw
            net.model[index].b -= lr * net.model[index].db
            index -= 2
        
    print("epoch: {}/{} loss: {}".format(epoch + 1, epochs, loss / len(X_train)))

    valid_correct = 0
    for i in range(len(X_valid)):
        net.forward(X_valid[i].reshape(-1, 1), MakeOneHot(10, y_valid[i]))

        if np.argmax(net.model[-1].p) == y_valid[i]:
            valid_correct += 1
    
    acc = valid_correct / len(X_valid)
    print("Valid Acc: {}".format(acc))
    if best_acc < acc:
        best_acc = acc
        best_model = copy.deepcopy(model)
        print("get new model!")

test_net = DNN(best_model)
test_correct = 0
for i in range(len(X_test)):
    test_net.forward(X_test[i].reshape(-1, 1), MakeOneHot(10, y_test[i]))

    if np.argmax(net.model[-1].p) == y_test[i]:
        test_correct += 1
print("Test Acc: {}".format(test_correct / len(X_test)))

epoch: 1/2000 loss: 0.9719739970709323
Valid Acc: 0.5525
get new model!
epoch: 2/2000 loss: 0.9644852901209782
Valid Acc: 0.54875
epoch: 3/2000 loss: 0.964521588153757
Valid Acc: 0.5475
epoch: 4/2000 loss: 0.9565311728578101
Valid Acc: 0.55125
epoch: 5/2000 loss: 0.9637373645079998
Valid Acc: 0.55375
get new model!
epoch: 6/2000 loss: 0.9677085163409623
Valid Acc: 0.55
epoch: 7/2000 loss: 0.9704112460428719
Valid Acc: 0.545
epoch: 8/2000 loss: 0.9488908244564405
Valid Acc: 0.54625
epoch: 9/2000 loss: 0.9483549883677501
Valid Acc: 0.54875
epoch: 10/2000 loss: 0.9409913128581077
Valid Acc: 0.55875
get new model!
epoch: 11/2000 loss: 0.9525096523437421
Valid Acc: 0.55
epoch: 12/2000 loss: 0.9420394918082762
Valid Acc: 0.54625
epoch: 13/2000 loss: 0.9399991973543196
Valid Acc: 0.54875
epoch: 14/2000 loss: 0.9349830589166966
Valid Acc: 0.54875
epoch: 15/2000 loss: 0.9441376899383824
Valid Acc: 0.55625
epoch: 16/2000 loss: 0.9412147023382906
Valid Acc: 0.55375
epoch: 17/2000 loss: 0.93954378

Valid Acc: 0.6475
epoch: 138/2000 loss: 0.7364042955708829
Valid Acc: 0.63625
epoch: 139/2000 loss: 0.7504676847112403
Valid Acc: 0.645
epoch: 140/2000 loss: 0.7378452954344088
Valid Acc: 0.6475
epoch: 141/2000 loss: 0.7351953867721431
Valid Acc: 0.64875
epoch: 142/2000 loss: 0.7724639653105091
Valid Acc: 0.635
epoch: 143/2000 loss: 0.7446271725531538
Valid Acc: 0.64125
epoch: 144/2000 loss: 0.7363386015468582
Valid Acc: 0.63875
epoch: 145/2000 loss: 0.7344064907989056
Valid Acc: 0.63875
epoch: 146/2000 loss: 0.7354445198343079
Valid Acc: 0.64375
epoch: 147/2000 loss: 0.7269664916216656
Valid Acc: 0.64375
epoch: 148/2000 loss: 0.7242695896004588
Valid Acc: 0.64
epoch: 149/2000 loss: 0.7228980751685323
Valid Acc: 0.63625
epoch: 150/2000 loss: 0.7447566059906218
Valid Acc: 0.64375
epoch: 151/2000 loss: 0.7361572621688539
Valid Acc: 0.635
epoch: 152/2000 loss: 0.7388229045517519
Valid Acc: 0.63625
epoch: 153/2000 loss: 0.7208898133272615
Valid Acc: 0.63625
epoch: 154/2000 loss: 0.74743193

epoch: 274/2000 loss: 0.6443241244164181
Valid Acc: 0.6675
epoch: 275/2000 loss: 0.6461029430432003
Valid Acc: 0.6725
epoch: 276/2000 loss: 0.6457149828750165
Valid Acc: 0.675
epoch: 277/2000 loss: 0.6448767929371603
Valid Acc: 0.67375
epoch: 278/2000 loss: 0.6457644689349678
Valid Acc: 0.66375
epoch: 279/2000 loss: 0.6345031886235183
Valid Acc: 0.665
epoch: 280/2000 loss: 0.6378999601939931
Valid Acc: 0.665
epoch: 281/2000 loss: 0.6403947413943074
Valid Acc: 0.6625
epoch: 282/2000 loss: 0.6321525822411984
Valid Acc: 0.66
epoch: 283/2000 loss: 0.6410992966193387
Valid Acc: 0.67
epoch: 284/2000 loss: 0.6390345890143996
Valid Acc: 0.6725
epoch: 285/2000 loss: 0.6375676173225568
Valid Acc: 0.66125
epoch: 286/2000 loss: 0.6356062069555205
Valid Acc: 0.675
epoch: 287/2000 loss: 0.6267732339088568
Valid Acc: 0.67875
epoch: 288/2000 loss: 0.6280258621492494
Valid Acc: 0.6775
epoch: 289/2000 loss: 0.6259823811675077
Valid Acc: 0.675
epoch: 290/2000 loss: 0.6263857014832371
Valid Acc: 0.67375
e

epoch: 412/2000 loss: 0.5605231376073234
Valid Acc: 0.67
epoch: 413/2000 loss: 0.5588180160501018
Valid Acc: 0.67375
epoch: 414/2000 loss: 0.5560027068005444
Valid Acc: 0.685
epoch: 415/2000 loss: 0.5533613144450901
Valid Acc: 0.67625
epoch: 416/2000 loss: 0.5510844263720353
Valid Acc: 0.67625
epoch: 417/2000 loss: 0.5532852161930282
Valid Acc: 0.675
epoch: 418/2000 loss: 0.5537013084593545
Valid Acc: 0.675
epoch: 419/2000 loss: 0.5547044272363751
Valid Acc: 0.6775
epoch: 420/2000 loss: 0.551101669698898
Valid Acc: 0.6775
epoch: 421/2000 loss: 0.5514465056968134
Valid Acc: 0.67625
epoch: 422/2000 loss: 0.5573606009891282
Valid Acc: 0.675
epoch: 423/2000 loss: 0.579023330090395
Valid Acc: 0.6825
epoch: 424/2000 loss: 0.5756801618362188
Valid Acc: 0.68125
epoch: 425/2000 loss: 0.5660398098092135
Valid Acc: 0.6775
epoch: 426/2000 loss: 0.5606121816203231
Valid Acc: 0.675
epoch: 427/2000 loss: 0.5608971878196951
Valid Acc: 0.67875
epoch: 428/2000 loss: 0.5563354889017851
Valid Acc: 0.68
ep

epoch: 550/2000 loss: 0.5122080690646016
Valid Acc: 0.6975
epoch: 551/2000 loss: 0.5113947171340102
Valid Acc: 0.69875
epoch: 552/2000 loss: 0.512078274666878
Valid Acc: 0.70375
epoch: 553/2000 loss: 0.5180256109952014
Valid Acc: 0.69875
epoch: 554/2000 loss: 0.5151879417516793
Valid Acc: 0.695
epoch: 555/2000 loss: 0.515160763964828
Valid Acc: 0.69375
epoch: 556/2000 loss: 0.5208601744292972
Valid Acc: 0.6975
epoch: 557/2000 loss: 0.5173369444769002
Valid Acc: 0.69875
epoch: 558/2000 loss: 0.5111114040971054
Valid Acc: 0.6975
epoch: 559/2000 loss: 0.5123887761592447
Valid Acc: 0.70125
epoch: 560/2000 loss: 0.5124749083922593
Valid Acc: 0.70125
epoch: 561/2000 loss: 0.5102688295975669
Valid Acc: 0.70125
epoch: 562/2000 loss: 0.5100308798668234
Valid Acc: 0.70125
epoch: 563/2000 loss: 0.5090597009655259
Valid Acc: 0.7
epoch: 564/2000 loss: 0.5097483220223572
Valid Acc: 0.70125
epoch: 565/2000 loss: 0.5140139679987566
Valid Acc: 0.7025
epoch: 566/2000 loss: 0.5188115099345061
Valid Acc: 

...

epoch: 1372/2000 loss: 0.38850014301116054
Valid Acc: 0.7525
epoch: 1373/2000 loss: 0.38957690568238257
Valid Acc: 0.75375
get new model!
epoch: 1374/2000 loss: 0.38738974110664215
Valid Acc: 0.755
get new model!
epoch: 1375/2000 loss: 0.3855102237904577
Valid Acc: 0.74625
epoch: 1376/2000 loss: 0.3987726610136577
Valid Acc: 0.755
epoch: 1377/2000 loss: 0.3971691357732736
Valid Acc: 0.74
epoch: 1378/2000 loss: 0.39889874337607417
Valid Acc: 0.73625
epoch: 1379/2000 loss: 0.4129714808364517
Valid Acc: 0.73875
epoch: 1380/2000 loss: 0.39985978935691047
Valid Acc: 0.7375
epoch: 1381/2000 loss: 0.3981986708845397
Valid Acc: 0.7425
epoch: 1382/2000 loss: 0.3888821738529144
Valid Acc: 0.7425
epoch: 1383/2000 loss: 0.388958311152641
Valid Acc: 0.7375
epoch: 1384/2000 loss: 0.3917121965087639
Valid Acc: 0.74125
epoch: 1385/2000 loss: 0.39485289494230846
Valid Acc: 0.74
epoch: 1386/2000 loss: 0.3948925689189297
Valid Acc: 0.745
epoch: 1387/2000 loss: 0.41677487357221943
Valid Acc: 0.74375
epoch

Valid Acc: 0.7625
epoch: 1506/2000 loss: 0.3725099348630345
Valid Acc: 0.7625
epoch: 1507/2000 loss: 0.3973426710483594
Valid Acc: 0.74375
epoch: 1508/2000 loss: 0.39651910685133396
Valid Acc: 0.75
epoch: 1509/2000 loss: 0.3824583776343131
Valid Acc: 0.7525
epoch: 1510/2000 loss: 0.38176119482159115
Valid Acc: 0.7525
epoch: 1511/2000 loss: 0.37585470313306346
Valid Acc: 0.75625
epoch: 1512/2000 loss: 0.3730074415659981
Valid Acc: 0.7575
epoch: 1513/2000 loss: 0.37491311776557923
Valid Acc: 0.75625
epoch: 1514/2000 loss: 0.3787567872733474
Valid Acc: 0.7575
epoch: 1515/2000 loss: 0.37217301291759264
Valid Acc: 0.7575
epoch: 1516/2000 loss: 0.3700988218559234
Valid Acc: 0.7575
epoch: 1517/2000 loss: 0.3684729686569327
Valid Acc: 0.75875
epoch: 1518/2000 loss: 0.3691975768831029
Valid Acc: 0.7575
epoch: 1519/2000 loss: 0.37269305582658313
Valid Acc: 0.75875
epoch: 1520/2000 loss: 0.36797464619602527
Valid Acc: 0.76125
epoch: 1521/2000 loss: 0.3747426853965166
Valid Acc: 0.75875
epoch: 152

Valid Acc: 0.76625
epoch: 1640/2000 loss: 0.3491624401447753
Valid Acc: 0.7625
epoch: 1641/2000 loss: 0.34594268230920894
Valid Acc: 0.765
epoch: 1642/2000 loss: 0.3480496039773226
Valid Acc: 0.7625
epoch: 1643/2000 loss: 0.3489784449263997
Valid Acc: 0.765
epoch: 1644/2000 loss: 0.35869360818554985
Valid Acc: 0.765
epoch: 1645/2000 loss: 0.35269624835094177
Valid Acc: 0.76875
epoch: 1646/2000 loss: 0.34553777527081936
Valid Acc: 0.7675
epoch: 1647/2000 loss: 0.3465184521743418
Valid Acc: 0.7675
epoch: 1648/2000 loss: 0.34665716259536405
Valid Acc: 0.7675
epoch: 1649/2000 loss: 0.34522229875867533
Valid Acc: 0.7675
epoch: 1650/2000 loss: 0.3443294004569875
Valid Acc: 0.76375
epoch: 1651/2000 loss: 0.3442462200855755
Valid Acc: 0.765
epoch: 1652/2000 loss: 0.342923662338658
Valid Acc: 0.76875
epoch: 1653/2000 loss: 0.34366443958179155
Valid Acc: 0.77125
epoch: 1654/2000 loss: 0.3441000123638917
Valid Acc: 0.7675
epoch: 1655/2000 loss: 0.3426924737242999
Valid Acc: 0.76875
epoch: 1656/20

Valid Acc: 0.76625
epoch: 1775/2000 loss: 0.32495285668948365
Valid Acc: 0.76625
epoch: 1776/2000 loss: 0.3249068809357578
Valid Acc: 0.765
epoch: 1777/2000 loss: 0.32671706784158133
Valid Acc: 0.765
epoch: 1778/2000 loss: 0.32542307581722574
Valid Acc: 0.76625
epoch: 1779/2000 loss: 0.32774217094819413
Valid Acc: 0.7675
epoch: 1780/2000 loss: 0.3263678062678679
Valid Acc: 0.76875
epoch: 1781/2000 loss: 0.32539293585798446
Valid Acc: 0.7675
epoch: 1782/2000 loss: 0.32685487965187393
Valid Acc: 0.76875
epoch: 1783/2000 loss: 0.3275752950659833
Valid Acc: 0.7675
epoch: 1784/2000 loss: 0.3257285663180939
Valid Acc: 0.765
epoch: 1785/2000 loss: 0.3241082559659777
Valid Acc: 0.76625
epoch: 1786/2000 loss: 0.3359712390643035
Valid Acc: 0.77
epoch: 1787/2000 loss: 0.33219809434526854
Valid Acc: 0.77
epoch: 1788/2000 loss: 0.3293526816514969
Valid Acc: 0.76625
epoch: 1789/2000 loss: 0.32638191675912787
Valid Acc: 0.765
epoch: 1790/2000 loss: 0.3243083046270111
Valid Acc: 0.7675
epoch: 1791/200

Valid Acc: 0.78625
epoch: 1909/2000 loss: 0.31621883061049855
Valid Acc: 0.7875
epoch: 1910/2000 loss: 0.315296472572337
Valid Acc: 0.785
epoch: 1911/2000 loss: 0.31415055304644673
Valid Acc: 0.78625
epoch: 1912/2000 loss: 0.30833055770027684
Valid Acc: 0.785
epoch: 1913/2000 loss: 0.30942747665395354
Valid Acc: 0.78375
epoch: 1914/2000 loss: 0.3098873882572012
Valid Acc: 0.7875
epoch: 1915/2000 loss: 0.3168079780219671
Valid Acc: 0.7925
get new model!
epoch: 1916/2000 loss: 0.3230693699913602
Valid Acc: 0.7825
epoch: 1917/2000 loss: 0.31311178945483065
Valid Acc: 0.78375
epoch: 1918/2000 loss: 0.3117708905201131
Valid Acc: 0.78375
epoch: 1919/2000 loss: 0.31068142065630405
Valid Acc: 0.7875
epoch: 1920/2000 loss: 0.3067289088234111
Valid Acc: 0.785
epoch: 1921/2000 loss: 0.30581611327734753
Valid Acc: 0.78375
epoch: 1922/2000 loss: 0.30709652667290416
Valid Acc: 0.7825
epoch: 1923/2000 loss: 0.31146364104742535
Valid Acc: 0.78625
epoch: 1924/2000 loss: 0.30585564363226697
Valid Acc: 0